In [ ]:
#loading libraries 
import pandas as pd
import numpy as np
import time
import tweepy
import re
import json
import os 

In [ ]:
#gives us the files in the directory 
directory= '.'
for filename in os.listdir(directory):
    print(os.path.join(directory, filename))

In [ ]:
#gives us the files in the directory that end with .json
directory= '.'
for filename in os.listdir(directory):
    if filename.endswith(".json"):
        print(os.path.join(directory, filename))

In [ ]:
#read the json files and append them 
directory= '.'
covid=pd.DataFrame()

for filename in os.listdir(directory):
    if filename.endswith(".json"):
        #address=os.path.join(directory, filename)
        with open(filename, 'r') as myfile:
            file=myfile.read()
            obj=json.loads(file)
            covid=covid.append(pd.DataFrame(obj))
        

In [ ]:
#save the file into one json file 
covid.to_json('./covid_tweets_clean.json', orient='records')

In [ ]:
with open('covid_tweets_clean.json', 'r') as myfile:
    data=myfile.read()
    
obj= json.loads(data)
covid_tweets_clean=pd.DataFrame(obj)

In [ ]:
#covid_tweets_clean.head()
#covid_tweets_clean.info()

In [ ]:
#cleaning the tweets
def remove_pattern(input_txt, pattern):
    r = re.findall(pattern, input_txt)
    for i in r:
        input_txt = re.sub(i, '', input_txt)        
    return input_txt
def clean_tweets(tweets):
    #remove twitter Return handles (RT @xxx:)
    tweets = np.vectorize(remove_pattern)(tweets, "RT @[\w]*:") 
    
    #remove twitter handles (@xxx)
    tweets = np.vectorize(remove_pattern)(tweets, "@[\w]*")
    
    #remove URL links (httpxxx)
    tweets = np.vectorize(remove_pattern)(tweets, "https?://[A-Za-z0-9./]*")
    
    #remove special characters, numbers, punctuations (except for #)
    tweets = np.core.defchararray.replace(tweets, "[^a-zA-Z]", " ")
    
    
    return tweets

In [ ]:
#give us the cleaned up tweets
covid_tweets_clean['text'] = clean_tweets(covid_tweets_clean['text'])
covid_tweets_clean.head()

In [ ]:
#get rid of qoutation marks 
covid_tweets_clean['location']= covid_tweets_clean['location'].str.replace('"', '')

In [ ]:
#replace 'united states' with 'usa' just to be consistent 
covid_tweets_clean['location']=covid_tweets_clean['location'].str.replace('United States', 'USA')